In [1]:
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.library.parameters import SeasonTypeAllStar
import pandas as pd
import time

# --- Settings ---
SEASONS = ["2023-24", "2024-25"]   # last two seasons
SEASON_TYPE = SeasonTypeAllStar.regular  # "Regular Season"
TOP_N = 150
OUTPUT_BASENAME = "nba_top150_per_game"  # will create one CSV per season + a combined file

# Columns to keep (per-game)
KEEP_COLS = [
    "SEASON", "PLAYER_ID", "PLAYER_NAME", "TEAM_ABBREVIATION", "GP", "MIN",
    "PTS", "REB", "AST", "STL", "BLK", "TOV", "FG3M", "FG_PCT", "FT_PCT"
]

def fetch_season_per_game(season: str) -> pd.DataFrame:
    # Query per-game stats for the season
    res = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season,
        season_type_all_star=SEASON_TYPE,
        per_mode_detailed="PerGame",  # per-game averages
        measure_type_detailed_defense="Base",  # the basic box score set
        pace_adjust="N"  # raw per-game, not pace-adjusted
    )
    df = res.get_data_frames()[0].copy()
    df["SEASON"] = season

    # Keep only the fields you want (renaming 3-pointers to your wording later if you like)
    df = df[KEEP_COLS]

    # Rank players by total minutes (GP * MIN) to get the most-regular players
    df["TOTAL_MIN"] = df["GP"] * df["MIN"]
    df = df.sort_values("TOTAL_MIN", ascending=False).head(TOP_N).drop(columns=["TOTAL_MIN"])

    # (Optional) round percentages to 3 decimals
    for col in ["FG_PCT", "FT_PCT"]:
        df[col] = df[col].round(3)

    # (Optional) rename for human-friendliness
    df = df.rename(columns={
        "FG3M": "3PM",        # 3-pointers made per game
        "FG_PCT": "FG%",
        "FT_PCT": "FT%"
    })
    return df

def main():
    all_frames = []
    for i, season in enumerate(SEASONS):
        df = fetch_season_per_game(season)
        df.to_csv(f"{OUTPUT_BASENAME}_{season}.csv", index=False)
        all_frames.append(df)
        # Be kind to the API
        if i < len(SEASONS) - 1:
            time.sleep(1.2)

    combined = pd.concat(all_frames, ignore_index=True)
    combined.to_csv(f"{OUTPUT_BASENAME}_combined.csv", index=False)
    print("Done:")
    for season in SEASONS:
        print(f" - {OUTPUT_BASENAME}_{season}.csv")
    print(f" - {OUTPUT_BASENAME}_combined.csv")

if __name__ == "__main__":
    main()


Done:
 - nba_top150_per_game_2023-24.csv
 - nba_top150_per_game_2024-25.csv
 - nba_top150_per_game_combined.csv
